In [1]:
#debug options
# http://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
#incleding modules
import pandas as pd
from os import path
import os
import utils.shared as utils
import utils.preprocessing as preprocessing
import utils.analysis as analysis
import matplotlib.pyplot as plt

In [33]:
# Path variable initialize
root_dir = '../rawdata/sber/'
xlsx_file = path.join(root_dir,'tech_suply.xlsx')
# xlsx_file = path.join(root_dir,'toy.xlsx')
save_dir = '../preprodata/stage4/sber/'
# os.mkdir(save_dir)
cor_train = path.join(save_dir,'train.cor')
man_train = path.join(save_dir,'train.man')
cor_test = path.join(save_dir,'test.cor')
man_test= path.join(save_dir,'test.man')
cor_dev_test = path.join(save_dir,'dev_test.cor')
man_dev_test = path.join(save_dir,'dev_test.man')
cor_all = path.join(save_dir,'used_text.cor')
man_all = path.join(save_dir,'used_text.man')
man_voc = path.join(save_dir,'vocab.man')
cor_voc = path.join(save_dir,'vocab.cor')

# Addition variable initialize
part_of_all_data = 1
train_part_of_used_data = 0.95
dev_part_of_test = 0.5
pad_enable = False
label_line_len = 140

In [4]:
# Get dataset xlsx => df
rawdataset = pd.read_excel(xlsx_file)

In [5]:
# Drop useless columns df => df
l_column ="CHAT_ID"
r_column ="TYPE"
dataset = rawdataset.loc[:, l_column:r_column]

In [6]:
# Tokinaze useless columns df => df
column_name = 'TEXT'
patterns = utils.regex_patterns.std_tokinaze
tokinazed_dataset = utils.tokinaze_column(dataset, column_name,patterns)

100% (5 of 5) |###########################| Elapsed Time: 0:00:25 Time: 0:00:25


In [7]:
# Get chats list df => [df0,df1,..]
column_name = 'CHAT_ID'
chats = preprocessing.get_chats_list(tokinazed_dataset, column_name)

100% (58082 of 58082) |###################| Elapsed Time: 0:01:22 Time: 0:01:22


In [8]:
# Insert a special tag in dialogue begining  [df0,df1,..] => [df0,df1,..]
chats = preprocessing.pauses_insert_in_begin(chats)

100% (58082 of 58082) |###################| Elapsed Time: 0:01:13 Time: 0:01:13


In [9]:
# Insert a special tag between the replies of the same author [df0,df1,..] => [df0,df1,..]
chats = preprocessing.insert_pauses(chats)

100% (58082 of 58082) |###################| Elapsed Time: 0:09:30 Time: 0:09:30


In [10]:
# Delete a replica dublicates [df0,df1,..] => [df0,df1,..]
chats = preprocessing.delete_dublicates(chats)

100% (58082 of 58082) |###################| Elapsed Time: 0:05:11 Time: 0:05:11


In [11]:
# Concatenate near dialogs by one author [df0,df1,..] => [df0,df1,..]
chats = preprocessing.sentence_concatenate(chats)

100% (58062 of 58062) |###################| Elapsed Time: 0:02:46 Time: 0:02:46


In [12]:
# If end dialog is sended by corporate then delete it [df0,df1,..] => [df0,df1,..]
chats = preprocessing.delete_corp_endings(chats)

100% (58062 of 58062) |###################| Elapsed Time: 0:00:12 Time: 0:00:12


In [13]:
# Delete chats with replica numbel less count_trashhold [df0,df1,..] => [df0,df1,..]
chats = preprocessing.delete_small_chat(chats, count_trashhold=5)

100% (58062 of 58062) |###################| Elapsed Time: 0:00:00 Time: 0:00:00


In [14]:
# Delete chats with replica numbel less count_trashhold [df0,df1,..] => [df0,df1,..]
count_trashhold=10
chats = preprocessing.delete_small_chat(chats, count_trashhold)

100% (54493 of 54493) |###################| Elapsed Time: 0:00:00 Time: 0:00:00


In [15]:
# Add service tag in chats [df0,df1,..] => [df0,df1,..]
ser_taged_chats = preprocessing.add_service_tag(chats)

100% (31024 of 31024) |###################| Elapsed Time: 0:11:23 Time: 0:11:23


In [16]:
# Concatenate near dialogs by one author [df0,df1,..] => [df0,df1,..]
ser_taged_chats = preprocessing.sentence_concatenate(ser_taged_chats)

100% (31024 of 31024) |###################| Elapsed Time: 0:05:20 Time: 0:05:20


In [17]:
# Add indexes [df0,df1,..],[df0,df1,..] => [df0,df1,..]
context_len = 140
man_replic_max = 2
man_chats = preprocessing.add_indexing(ser_taged_chats,chats,context_len,man_replic_max)

| 31023 Elapsed Time: 0:03:07                                                  


In [18]:
# Zip chats [df0,df1,..],[df0,df1,..] => [(df0,df0),(df1,df1),..]
context_chats = ser_taged_chats
ans_chats = man_chats
resulted_chats = preprocessing.zipper(context_chats, ans_chats)

In [44]:
used_df, _ = preprocessing.rand_split_list(resulted_chats,part_of_all_data)
train_df, test_df = preprocessing.rand_split_list(used_df,train_part_of_used_data)
dev_test, test = preprocessing.rand_split_list(test_df,dev_part_of_test)
preprocessing.dataset_make(train_df,cor_train,man_train,pad_enable,label_line_len)
preprocessing.dataset_make(dev_test,cor_dev_test,man_dev_test,pad_enable,label_line_len)
preprocessing.dataset_make(test,cor_test,man_test,pad_enable,label_line_len)
preprocessing.dataset_make(used_df,cor_all,man_all,pad_enable,label_line_len)

  0% (47 of 29472) |                      | Elapsed Time: 0:00:00 ETA:  0:06:39


                        Catch empty line:
                        train_line: 
                        label_line: Вы вошли обратно в систему ?
                        

                        Catch empty line:
                        train_line: 
                        label_line: Спасибо за ожидание , уточнила информацию по Вашему вопросу .
                        


  0% (90 of 29472) |                      | Elapsed Time: 0:00:01 ETA:  0:05:46


                        Catch empty line:
                        train_line: 
                        label_line: Вы можете оставить жалобу на работу программы на Линии качества по телефону 8-800-555-57-77 по кнопке " 3 " или на сайте sberbank . ru в разделе " Обратная связь " , далее нажмите " Напишите нам " и в разделе " Укажите , с чем связано Ваше обращение " выбрать " Качество обслуживания " .
                        


  0% (114 of 29472) |                     | Elapsed Time: 0:00:01 ETA:  0:05:26

...

 94% (29228 of 31024) |#################  | Elapsed Time: 0:01:41 ETA:  0:00:04


                        Catch empty line:
                        train_line: 
                        label_line: Служба технической поддержки Сбербанка , меня зовут Надежда . Здравствуйте !
                        


 99% (30930 of 31024) |################## | Elapsed Time: 0:01:44 ETA:  0:00:00


                        Catch empty line:
                        train_line: 
                        label_line: Служба технической поддержки Сбербанка , меня зовут Оксана . Здравствуйте !
                        


100% (31024 of 31024) |###################| Elapsed Time: 0:01:45 Time: 0:01:45


In [45]:
cor_vocab = preprocessing.mkvocab(cor_all)
man_vocab = preprocessing.mkvocab(man_all)
preprocessing.save_vocab(man_vocab, man_voc)
preprocessing.save_vocab(cor_vocab, cor_voc)

| 275326 Elapsed Time: 0:00:04                                                 
| 275326 Elapsed Time: 0:00:01                                                 
100% (26918 of 26918) |###################| Elapsed Time: 0:00:00 Time: 0:00:00
100% (77178 of 77178) |###################| Elapsed Time: 0:00:00 Time: 0:00:00


In [39]:
len(test)

776

In [42]:
q

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
668243    False
668244    False
668245    False
668246    False
668247    False
668248    False
668249    False
668250    False
668251    False
668252    False
668253    False
668254    False
668255    False
668256    False
668257    False
668258    False
668259    False
668260    False
668261    False
668262    False
668263    False
668264    False
668265    False
668266    False
668267    False
668268    False
668269    False
668270    False
668271    False
668272    False
Name: TEXT, Length: 6682